# Streams and Teams

The purpose of this notebook is to take a signup sheet of people who want to join a 5 member team, select a team name and then create a Speckle stream for each team and invite each team member to their respective stream using the specklepy SDK.

## Setup

### Install the specklepy SDK

In [3]:
%%capture
# capture turns off the output for this cell which would just be the pip install log 🤫
%pip install specklepy
%pip install python-dotenv
%pip install gspread

%reload_ext dotenv
%dotenv

Get previously stored credentials from the .env file

In [4]:
import os
HOST_SERVER = os.getenv('HOST_SERVER')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')

We will use the Google Sheets API to read the signup sheet.

### Import the Google Sheets API

In [3]:
import gspread

Define the name of the sheet and the range of cells containing the team data:

In [12]:
sheet_name = "Team Signup Sheet"
worksheet_name = "Teams"
cell_range = "A2:E21"

We need to give the service account details to gspread, which can be done using the service_account method.

In [44]:
service_account_client = gspread.service_account(filename="speckle-examples-jupyter-8cb6c49b0d7c.json")

The path to the json file that contains the service credentials is passed to the filename parameter. If the file is in the same working directory as the notebook, you can just write the name of the file.

The `service_account_client` is a gspread client, which can be used for connecting to the sheets by using the open method and the sheet name.

In [45]:
sheet = service_account_client.open(sheet_name)

A Google sheet document might have multiple pages (i.e. worksheets) so we also need to specify the page name before getting the data. Ours has one page that is called `Teams`. We'll read the data from the range defined above.

In [48]:
work_sheet = sheet.worksheet(worksheet_name)

team_data = work_sheet.get_all_records() 

We can loop the `team_data` directly. Each row is a dictionary with the keys being the column names and the values being the values in the cells.

But first we'll need to import and authenticate the `specklepy` SDK.

In [5]:
from specklepy.api.client import SpeckleClient

client = SpeckleClient(host=HOST_SERVER)  # or whatever your host is
client.authenticate_with_token(ACCESS_TOKEN)  # or whatever your token is

client

SpeckleClient( server: https://speckle.xyz, authenticated: True )

Loop through the teams and create streams for each team if they don't already exist.

In [21]:
existing_streams = client.stream.list(stream_limit=100)

stream = existing_streams[0]

for n in range(1, 115):
    formatted_number = f"{n:02}"
    client.branch.create(stream.id, name=f"Team {formatted_number[0]}/{formatted_number[1:]}")


The next code block will delete existing branches. Be sure you want to do this before running it.

In [18]:
branches = client.branch.list(stream.id, branches_limit=100)

for n in range(0, len(branches)):
    client.branch.delete(stream.id,branches[n].id)

Now, for this example for all the teams, we'll create a stream and invite the team members to it.

In [ ]:

for team in team_data:
    # Extract the team information
    team_number = team["Team #"]
    team_name = team["Team Name"]
    team_members = [email.strip() for email in team["Members"].split(",")]

    # Construct the stream name
    stream_name = f"Team {team_number} - {team_name}"

    # Check if the stream already exists
    target_stream = None
    for stream in existing_streams:
        if stream.name == stream_name:
            target_stream = stream
            break

    if target_stream:
        print(f"Stream '{stream_name}' already exists with ID '{target_stream.id}'.")
    else:
        # Create the stream
        stream_id = client.stream.create(name=stream_name, description="Team stream for the special project.", is_public=False)
        print(f"Stream '{stream_name}' created with ID '{stream_id}'.")
        target_stream = client.stream.get(stream_id)

        # Add the stream to the list of existing streams
        existing_streams.append(target_stream)

    # Invite the team members to the stream
    message = f"""
{client.user.name} has invited you to join the stream '{target_stream.name}' on Speckle. 

You can view the stream here: {HOST_SERVER}/streams/{target_stream.id}
Please accept the invitation by clicking on the link in the email.
"""
    for member in team_members:
        invited = client.stream.invite(stream_id=target_stream.id, email=member, message=message)  
        print(f"{member} invited {invited}")

These functions all work by the individual, you may need to add a sleep timer to avoid hitting the API rate limit. If you are a server owner you can use the API for batch invite to avoid this.

In [ ]:
existing_streams = client.stream.list(stream_limit=100)

for team in team_data:
    # Extract the team information
    team_number = team["Team #"]
    team_name = team["Team Name"]
    team_members = [email.strip() for email in team["Members"].split(",")]

    # Construct the stream name
    stream_name = f"Team {team_number} - {team_name}"

    # Check if the stream already exists
    target_stream = None
    for stream in existing_streams:
        if stream.name == stream_name:
            target_stream = stream
            break

    if target_stream:
        print(f"Stream '{stream_name}' already exists with ID '{target_stream.id}'.")
    else:
        # Create the stream
        stream_id = client.stream.create(name=stream_name, description="Team stream for the special project.", is_public=False)
        print(f"Stream '{stream_name}' created with ID '{stream_id}'.")
        target_stream = client.stream.get(stream_id)

        # Add the stream to the list of existing streams
        existing_streams.append(target_stream)

    # Create the GraphQL query to use in the Explorer
    message = f"""
{client.user.name} has invited you to join the stream '{target_stream.name}' on Speckle. 

You can view the stream here: {HOST_SERVER}/streams/{target_stream.id}
Please accept the invitation by clicking on the link in the email.
"""
    for member in team_members:
        invited = client.stream.invite(stream_id=target_stream.id, email=member, message=message)  
        print(f"{member} invited {invited}")